In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet(r'C:/Users/ACER-PC/Desktop/Z/nyc dataset/fhv_tripdata_2021-01.parquet')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In [17]:
df.head(3)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.0
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.0


In [9]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime

In [19]:
df['duration'] = df['duration'].apply(lambda td : td.total_seconds() / 60)

# OR

#df['duration'] = df.duration.dt.total_seconds()/60



In [12]:
td = df['duration'].iloc[0]

In [15]:
td.total_seconds() / 60

17.0

In [18]:
df.duration.mean()

19.1672240937939

In [86]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction import DictVectorizer

In [50]:
#dropped values

drop_val = len(df) - len(df[(df.duration >= 1) & (df.duration <= 60)])

In [52]:
drop_val / len(df) * 100

3.8372359008484445

In [54]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [62]:
df['PUlocationID'].isna().sum() / len(df)

0.8352732770722617

In [81]:
#df[['PUlocationID', 'DOlocationID']] = df[['PUlocationID', 'DOlocationID']].fillna(-1).astype('int')

# ||

categorical = ['PUlocationID', 'DOlocationID']
df[categorical] =df[categorical].fillna(-1).astype('str')

In [71]:
df.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1,-1,None,B00009,17.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1,-1,None,B00009,17.0


In [85]:
#ONE HOT ENCODING

train_dicts = df[categorical].to_dict(orient='records')

In [87]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [94]:
y_train = df.duration.values

In [88]:
#TO FIND DIMENSIONALITY OF THE MATRIX

X_train.shape

#||

#len(dv.feature_names_)

(1109826, 525)

In [93]:
from sklearn.linear_model import LinearRegression

In [95]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [96]:
y_pred = lr.predict(X_train)

In [97]:
from sklearn.metrics import mean_squared_error

In [98]:
mean_squared_error(y_train, y_pred, squared=False)

10.528519107205753

In [101]:
categorical = ['PUlocationID', 'DOlocationID']
    
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df['duration'].apply(lambda td : td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df[categorical] =df[categorical].fillna(-1).astype('str')
    
    return df

In [118]:
df_val = read_data(r'C:/Users/ACER-PC/Desktop/Z/nyc dataset/fhv_tripdata_2021-02.parquet')

In [119]:
#convert to dict

val_dicts = df_val[categorical].to_dict(orient='records')

In [120]:
X_val = dv.transform(val_dicts)

In [121]:
y_pred = lr.predict(X_val)

In [122]:
y_val = df_val.duration.values

In [125]:
mean_squared_error(y_val, y_pred, squared=False)

12.8550870918208